<a href="https://colab.research.google.com/github/LucasNatalePires/kaggle_titanic/blob/main/titanic_version5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

In [10]:
#Getting to know our dataset
train = pd.read_csv("/content/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
test = pd.read_csv("/content/test.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


**This data processing process below, despite seeming long, is nothing more than all the treatments I did in the previous three versions, treating all null values, and transforming/creating columns so that the algorithm can have greater precision, more details in the respective versions:**

[Version 1](https://github.com/LucasNatalePires/kaggle_titanic/blob/main/titanic_version1.ipynb)

[Version 2](https://github.com/LucasNatalePires/kaggle_titanic/blob/main/titanic_version2.ipynb)

[Version 3](https://github.com/LucasNatalePires/kaggle_titanic/blob/main/titanic_version3.ipynb)

[Version 4](https://github.com/LucasNatalePires/kaggle_titanic/blob/main/titanic_version4.ipynb)

In [12]:
#Version 1

#Deleting those colums from train
train = train.drop(['Name','Ticket','Cabin'],axis=1)
#Deleting those colums from train
test = test.drop(['Name','Ticket','Cabin'],axis=1)



# Now it's time to deal with null values, starting with age
train_age = train.Age.mean()
test_age = test.Age.mean()
train.loc[train.Age.isnull(),'Age'] = train_age
test.loc[test.Age.isnull(),'Age'] = test_age


#Treating null values in the fare column
test_fare = test.Fare.mean()
test.loc[test.Fare.isnull(),'Fare'] = test_fare


#Treating null values in the embarked column
train_embarked = train.Embarked.mode()[0]
train.loc[train.Embarked.isnull(),'Embarked'] = train_embarked

In [13]:
#Version 2

#Function to check and change the 'female' data to '1', so I can work with more data in the model used.
train['MaleCheck'] = train.Sex.apply(lambda x:1 if x == 'male' else 0)

#Let's repeat the same thing with the datas from test
test['MaleCheck'] = test.Sex.apply(lambda x:1 if x == 'male' else 0)

**As I said previously about "work for nothing" or mistakes, in this function to check the passenger's sexuality, I initially checked if it was female, and yes, I would do the other steps. When I calculated the accuracy using the MLP Classifier, the value was 0.8135593220338984. Analyzing the code again, I replaced 'check female' with 'check male', despite a simple change, the accuracy rose to 0.8305084745762712**

In [14]:
#Version 3

#Import
from sklearn.preprocessing import RobustScaler
#Creating the scaler
transformer = RobustScaler().fit(train[['Age','Fare']])
#Transforming the datas
train[['Age', 'Fare']] = transformer.transform(train[['Age','Fare']])

#Creating the scaler
transformer = RobustScaler().fit(test[['Age','Fare']])
test[['Age', 'Fare']] = transformer.transform(test[['Age','Fare']])


#Create a function to check if the person is alone
def check_alone(x,y):
    if (x == 0 and y == 0):
        return 1
    else:
        return 0
#Create the column and apply the function
train['Alone'] = train.apply(lambda x:check_alone(x.SibSp, x.Parch), axis=1)
#Create the column and apply the function
test['Alone'] = test.apply(lambda x:check_alone(x.SibSp, x.Parch), axis=1)


#Creating the column 'relatives' on train
train['Relatives'] = train['SibSp'] + train['Parch']
#Doing the same thing on test
test['Relatives'] = test['SibSp'] + test['Parch']


#Treating the column 'Embarked'
#Import
from sklearn.preprocessing import OrdinalEncoder
#Creating the encoder
ports = ['S','C','Q']
enc = OrdinalEncoder(categories=[ports],dtype = 'int32')
#Fit with datas
enc = enc.fit(train[['Embarked']])
# Add these datas on the column train
train['Embarked'] = enc.transform(train[['Embarked']])
#The same job with test, Fit with datas
enc = enc.fit(test[['Embarked']])
# Add these datas on the column test
test['Embarked'] = enc.transform(test[['Embarked']])


#Delete the column 'Sex'
test = test.drop('Sex', axis=1)
train = train.drop('Sex',axis=1)

In [15]:
#View the treated dataset
train.head(3)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Alone,Relatives
0,1,0,3,-0.592240,1,0,-0.312011,0,1,0,1
1,2,1,1,0.638529,1,0,2.461242,1,0,0,1
2,3,1,3,-0.284548,0,0,-0.282777,0,0,1,0


**Separating the file in train and test, As this process has already been done several times, I will not focus on it again.**

**[train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)**

In [16]:
#Import
from sklearn.model_selection import train_test_split

#Separating the training base into X and y
X = train.drop(['PassengerId','Survived'], axis = 1)
y = train.Survived

#Separate train and validation
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.33, random_state=42)

**Based on the accuracy of the algorithms used, it was found that logistic regression had better performance, so I will not include KNC or Tree Decision, but I will include other algorithms, to see if there is any improvement**

**[Logistic regression](https://scikitlearn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)**

**[Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)**

**[MPL Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)**

**After that, let's import the [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) and see the results**

**I will check the [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) and finally, submit the predict**

In [17]:
from sklearn.linear_model import LogisticRegression

# Creating the classifier
clf_lr = LogisticRegression(random_state=42)

# Fit with data
clf_lr = clf_lr.fit(X_train,y_train)

# Making the predction
y_pred_lr = clf_lr.predict(X_validation)

In [18]:
#Import
from sklearn.ensemble import RandomForestClassifier

# Creating the Classifier
clf_rf = RandomForestClassifier(random_state=42)

# Fit with datas
clf_rf = clf_rf.fit(X_train, y_train)

# Making the predction
y_pred_rf = clf_rf.predict(X_validation)


In [19]:
#Import
from sklearn.neural_network import MLPClassifier

# Creating the Classifier
clf_mlp = MLPClassifier(random_state=42, max_iter= 5000)

# Fit with datas
clf_mlp = clf_mlp.fit(X_train, y_train)

# Making the predction
y_pred_mlp = clf_mlp.predict(X_validation)

In [20]:
#Import
from sklearn.metrics import accuracy_score
#Logistic Regression accuracy
accuracy_score(y_validation,y_pred_lr)

0.8067796610169492

In [21]:
# Random Forest accuracy
accuracy_score(y_validation, y_pred_rf)

0.7898305084745763

In [22]:
# mpl accuracy
accuracy_score(y_validation,y_pred_mlp)

0.8305084745762712

In [23]:
#Import
from sklearn.metrics import confusion_matrix

In [24]:
#logistic regression confusion_matrix
confusion_matrix(y_validation, y_pred_lr)

array([[152,  23],
       [ 34,  86]])

In [25]:
#random forest confusion_matrix
confusion_matrix(y_validation, y_pred_rf)

array([[148,  27],
       [ 35,  85]])

In [26]:
# mlp Classifier confusion_matrix
confusion_matrix(y_validation, y_pred_mlp)

array([[160,  15],
       [ 35,  85]])

**Until this part, nothing new. All of this was done in [version 4](https://github.com/LucasNatalePires/kaggle_titanic/blob/main/titanic_version4.ipynb). Now I will demonstrate how to deal with overfitting in this situation, basically improving the parameters. For this I will use [Grid Search Cv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to estimate the best parameters**

In [27]:
#Import
from sklearn.model_selection import train_test_split

#Separating the training base into X and y
X = train.drop(['PassengerId','Survived'], axis =1)
y = train.Survived

#Separate train and validation
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
#Import
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [29]:
#To ignore the warnings
import warnings

warnings.filterwarnings('ignore')

**I will use these algorithms to look for better parameters, consequently better results:**


**-- [Logistic regression](https://scikitlearn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)**

**-- [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)**

**-- [MPL Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)**




**Although the process seems similar to what we have done many times during the project, there is a big difference. Using [Grid Search CV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html), I plotted some parameters to get the best score**


**NOTE: The final value of the parameters was determined based on trial and error, a process that takes a little longer, but is necessary to improve accuracy. Every time I test a parameter, I see the results from best_params, and so I change the values**



**After that, let's import the [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) and see the results**

**I will check the [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) and finally, submit the predict**



**I will do respectively: Logistic Regression, Random Forest and Mlp Classifier**

In [30]:
#Function to check the time
from datetime import datetime

def hour():
    time = datetime.now()
    print(str(time.hour)+':'+str(time.minute)+':'+str(time.second))

**Out of curiosity, I created a function that takes the current time, runs the algorithm and sees the time again. From this it is possible to know how long it took to process the data, which is important information to take into account, depending on the project**

# Random Forest

In [31]:
#Import Random Forest
from sklearn.ensemble import RandomForestClassifier

# Creating the Classifier
clf_rf = RandomForestClassifier(random_state=42)

#Creating the parameters
parameters_rf = {
    'n_estimators': [100,250,500,1000],
    'criterion': ['gini','entropy','log_loss'],
    'max_depth': [2,4,6,8,None],
    'max_features': ['sqrt','log2',None]
}

In [32]:
# Using the K Fold to split the data and the Grid SearchCv to find the best parameters
#The function time() is just to count how long the data is processed

hour()
KFold_rf = KFold(n_splits=8, random_state=42, shuffle=True)
grid_search_rf = GridSearchCV(clf_rf, parameters_rf,scoring='accuracy',cv=KFold_rf)
grid_search_rf = grid_search_rf.fit(X_train,y_train)
hour()

17:14:37
17:36:48


In [33]:
#Checking the best parameters
grid_search_rf.best_params_

{'criterion': 'entropy',
 'max_depth': 6,
 'max_features': 'sqrt',
 'n_estimators': 100}

In [34]:
#Checking the best score
grid_search_rf.best_score_

0.8314606741573034

# MPL Classifier

In [35]:
#Import Mpl Classifier
from sklearn.neural_network import MLPClassifier

# Creating the Classifier
clf_mpl = MLPClassifier(random_state=42)

#Creating the parameters
parameters_mpl = {
    'solver': ['lbfgs','sgd','adam'],
    'alpha': [10.0**(-1),10.0**(-5),10.0**(-7),10.0**(-10)],
    'max_iter': [200,500,1000, 5000],
}

In [36]:
# Using the K Fold to split the data and the Grid SearchCv to find the best parameters
# The function time() is just to count how long the data is processed
hour()
KFold_mpl = KFold(n_splits=8, random_state=42, shuffle=True)
grid_search_mpl = GridSearchCV(clf_mpl, parameters_mpl,scoring='accuracy',cv=KFold_mpl)
grid_search_mpl = grid_search_mpl.fit(X_train,y_train)
hour()

17:36:49
17:51:21


In [37]:
# Checking the best parameters
grid_search_mpl.best_params_

{'alpha': 0.1, 'max_iter': 200, 'solver': 'adam'}

In [38]:
#Checking the best score
grid_search_mpl.best_score_

0.8174157303370786

# Logistic Regression

In [39]:
# Import
from sklearn.linear_model import LogisticRegression

# Creating the classifier
clf_lr = LogisticRegression(random_state=42)

# Creating the parameters
parameters_lr = {
    'penalty': ['l1','l2'],
    'C': [0.01,0.1,1,10],
    'solver': ['lbfgs','liblinear','saga'],
    'max_iter': [100,1000,5000,10000]
}

In [40]:
# Using the K Fold to split the data and the Grid SearchCv to find the best parameters
# The function time() is just to count how long the data is processed
hour()
KFold_lr = KFold(n_splits=8, random_state=42, shuffle=True)
grid_search_lr = GridSearchCV(clf_lr,parameters_lr,scoring='accuracy',cv=KFold_lr)
grid_search_lr = grid_search_lr.fit(X_train, y_train)
hour()

17:51:21
17:51:33


In [41]:
#Checking the best parameters
grid_search_lr.best_params_

{'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}

In [42]:
#Checking the best score
grid_search_lr.best_score_

0.8089887640449438

**After selecting the best parameters, I will calculate the accuracy and the confusion matrix, to define which model to use and then send it**

In [43]:
#Import
from sklearn.metrics import accuracy_score

In [44]:
#Predicting on validation data with each of the best models
#Logistic Regression
clf_selected_lr = grid_search_lr.best_estimator_
y_pred_lr = clf_selected_lr.predict(X_validation)

In [45]:
#Predicting on validation data with each of the best models
#Random Forest
clf_selected_rf = grid_search_rf.best_estimator_
y_pred_rf = clf_selected_rf.predict(X_validation)

In [46]:
#Predicting on validation data with each of the best models
#Mpl Classifier
clf_selected_mpl = grid_search_mpl.best_estimator_
y_pred_mpl = clf_selected_mpl.predict(X_validation)

In [47]:
# Logistic Regression Accuracy
accuracy_score(y_validation, y_pred_lr)

0.8044692737430168

In [48]:
# Random Forest Accuracy
accuracy_score(y_validation, y_pred_rf)

0.8100558659217877

In [49]:
#Mpl Classifier Accuracy
accuracy_score(y_validation, y_pred_mpl)

0.8100558659217877

In [50]:
#Import
from sklearn.metrics import confusion_matrix

In [51]:
#Confusion Matrix Logistic Regression
confusion_matrix(y_validation, y_pred_lr)

array([[91, 14],
       [21, 53]])

In [52]:
#Confusion Matrix Random Forest
confusion_matrix(y_validation, y_pred_rf)

array([[93, 12],
       [22, 52]])

In [53]:
#Confusion Matrix Mpl Classifier
confusion_matrix(y_validation, y_pred_mpl)

array([[92, 13],
       [21, 53]])

In [54]:
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Alone,Relatives
331,1,1.215452,0,0,0.608317,0,1,1,0
733,2,-0.515317,0,0,-0.062981,0,1,1,0
382,3,0.176991,0,0,-0.282777,0,1,1,0


In [55]:
test.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Alone,Relatives
0,892,3,0.331562,0,0,-0.280670,2,1,1,0
1,893,3,1.311954,1,0,-0.315800,0,0,0,1
2,894,2,2.488424,0,0,-0.201943,2,1,1,0


In [56]:
#Deleting PassengerId from Test because test and training must have the same columns
X_test = test.drop(['PassengerId'], axis=1)

In [57]:
#Selecting the best model to our test data
y_pred = clf_selected_rf.predict(X_test)

In [58]:
#Creating the colum 'Survived'
test['Survived'] = y_pred

In [59]:
#Selecting just the colums 'PassengerId'and 'Survived' to send and the dataset updated
result = test[['PassengerId','Survived']]

In [60]:
#Exporting to .CSV
result.to_csv('result4.csv', index = False)